In [ ]:
import pandas as pd
import psycopg2
import json
from pandas.io.json import json_normalize
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Drugs dataset

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
# Loading in the drugs.csv from AWS
drugs

In [ ]:
# NOTE: There are some issues with unfurling the openfda column, specifically

# Unfurl the 'openfda' column
drugs['openfda'] = drugs['openfda'].apply(lambda x: json.loads(x) if pd.notnull(x) and isinstance(x, str) else {})

# Unfurl the nested JSON in the 'openfda' column and hold the index
df_openfda = pd.json_normalize(drugs['openfda'])
df_openfda.columns = [f"openfda_{col}" for col in df_openfda.columns]  # Rename to avoid collisions

# Concatenate the expanded openfda data back with the original dataframe, retaining the index
df_with_fully_expanded = pd.concat([drugs.drop(columns=['openfda']), df_openfda], axis=1)
df_with_fully_expanded

In [ ]:
# This is somewhat custom for now. A lot of these columns have nested json. 
# Some we care about like "active_ingredients", others we don't really need to worry about.

def explode_json_columns(drugs):
    
    # Start with products
    drugs['products'] = drugs['products'].apply(lambda x: json.loads(x) if isinstance(x, str) else [])
    df_exploded = drugs.explode('products').reset_index(drop=True)
    df_products_expanded = pd.json_normalize(df_exploded['products'])
    df_products = pd.concat([df_exploded.drop(columns=['products']), df_products_expanded], axis=1)
    
    # Now active_ingredients list
    df_products['active'] = df_products['active_ingredients'].apply(lambda x: json.loads(x) if isinstance(x, str) else [])
    df_exploded = df_products.explode('active_ingredients').reset_index(drop=True)
    df_products_expanded = pd.json_normalize(df_exploded['active_ingredients'])
    df_active = pd.concat([df_exploded.drop(columns=['active_ingredients', 'active']), df_products_expanded], axis=1)
    df_active = df_active.rename(columns={'name': 'active_ingredients', 'strength': 'strength_active_ingredients'})
    
    # finally, explode submissions column
    df_active['submissions'] = df_active['submissions'].apply(lambda x: json.loads(x) if isinstance(x, str) else [])
    df_exploded = df_active.explode('submissions').reset_index(drop=True)
    df_submissions_expanded = pd.json_normalize(df_exploded['submissions'])
    df_submissions = pd.concat([df_exploded.drop(columns=['submissions']), df_submissions_expanded], axis=1)

    return df_submissions

In [ ]:
drugs_exploded = explode_json_columns(df_with_fully_expanded)

In [ ]:
# Fix dates in submission_status_date

drugs_exploded['submission_status_date'] = pd.to_datetime(drugs_exploded['submission_status_date']) 

In [ ]:
drugs_exploded['application_number'].nunique()

In [ ]:
# You can see here we don't really care about the "application_docs" column
# The reason this has duplicates is because it has different submission dates/numbers. 
# I plan to do some research on submission_type == 'ORIG' and submission_status_date so I can ensure 
# I get the original approval date. Also, note you may not be able to just throw away the other submissions as
# they may signify updated materials in other columns. For drug formulation v1, this is less of a concern.


drugs_exploded[drugs_exploded['application_number'] == 'ANDA076204']

# Product Labels Dataset

In [ ]:
product_labels.head(10)

In [ ]:
# First drop all columns that end in _table because they're just the HTML behind the table in the label insert
product_labels = product_labels.loc[:, ~product_labels.columns.str.endswith('_table')]
product_labels.head(10)

## indications_and_usage

In [ ]:
# Clean indications and usage column

product_label_test = product_labels[['id', 'openfda', 'indications_and_usage']]

product_label_test['indications_and_usage'] = product_label_test['indications_and_usage'].astype(str)

# First change all text to lowercase
def text_lower(text):
    text = text.lower()
    return text

product_label_test['indications_and_usage_lower'] = product_label_test['indications_and_usage'].apply(lambda x: text_lower(x))

# remove punctuation
punctuation = '!"#$&\'()*+-:;<=>?@[\\]^_`{|}~1234567890.'
def remove_punct(text):
    nopunct = "".join([char for char in text if char not in punctuation])
    return nopunct

product_label_test['indications_and_usage_lower'] = product_label_test['indications_and_usage_lower'].apply(lambda x: remove_punct(x))


# now tokenize the text
def token(text):
    t = WordPunctTokenizer().tokenize(text)
    return t

product_label_test['indications_and_usage_token'] = product_label_test['indications_and_usage_lower'].apply(lambda x: token(x))


# remove stopwords
#nltk.download('stopwords')
#stopword = stopwords.words('english')
#more_stopwords = ["indications", "usage", "1", "use", "used", "directed", "nan"]
#stopword.extend(more_stopwords)


# Keep adding to stopwords to see if we can get this to a good place
stopword = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", "indications", "usage", "1", 
            "use", "used", "directed", "nan", "direction", "help", "reduce", "prevent", "cause", "potentially", "us", 
           "may", "see", "clinical", "indicated", "treatment", "treat", "treated", "study", "tablet", 
           "years", "age", "patients", "year", "patient", "usp", "temporarily", "relief", "caused", "older", 
           "information", "temporary", "hour", "approved", "uses", "relieved", "relieve", "symptoms", "symptom", "associated", 
           "controlled", "control", "trial", "trials", "trialed", "helps", "helped", "help", 
           "extended", "release", "tablet", "available", "measure", "measures", "measured", "direction", "directions", 
           "adult", "soap", "water", "physician", "physicians", "factor", "factors", "dosage administration", "absolute", 
           "structure", "structures", "structured", "adjunctive therapy", "adjuvant therapy", "data", "local", "would", "expect", "expects", 
           "expected", "wide", "variety", "varieties", "also", "seen", "saw", "make", "made"]

def remove_stopwords(tokenized_text):
    text = [word for word in tokenized_text if word not in stopword]
    return text

product_label_test['indications_and_usage_nostopwords'] = product_label_test['indications_and_usage_token'].apply(lambda x: remove_stopwords(x))


# need to lemmatize 
nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [lemmatizer.lemmatize(word) for word in tokenized_text]
    return text

product_label_test['indications_and_usage_lemma'] = product_label_test['indications_and_usage_nostopwords'].apply(lambda x: lemmatizing(x))


product_label_test['indications_and_usage_clean'] = product_label_test['indications_and_usage_lemma'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

product_label_test

In [ ]:
# See if wordcloud helps us refine stopwords a bit


%matplotlib inline
result = product_label_test['indications_and_usage_clean'].astype('str')

result = Counter(result)

text = product_label_test['indications_and_usage_clean'].astype('str')

print ("There are", text.nunique(), "unique indications in this dataset given", 
       product_label_test['id'].nunique(), "drugs.")

stopwords = ['none']

wordcloud = WordCloud(stopwords = stopwords, background_color = "white", max_font_size = 300, random_state = 42, 
                     width = 800, height = 500).generate_from_frequencies(result)

plt.figure(figsize=[25,20])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Let's get some indication counts so we can take a look at how clean our data is

text = product_label_test['indications_and_usage_clean'].astype('str') + ' '
text = ' '.join(review for review in product_label_test.indications_and_usage_clean)


counts = WordCloud(stopwords = stopwords).process_text(text)
indication_counts = pd.Series(counts).to_frame().sort_values([0], ascending = False)
indication_counts.reset_index(level=0, inplace = True)
indication_counts = indication_counts.rename(columns={0:'count', 'index':'indication'})
indication_counts

In [ ]:
pd.set_option('display.max_rows', None)
indication_counts

# ChatGPT approach

In [ ]:
indications = pd.read_csv('/Users/helena/Downloads/5000_drug_indications.csv')
indications

In [ ]:
product_label_test_gpt = product_labels[['id', 'openfda', 'indications_and_usage']]

In [ ]:
# First change all text to lowercase
def text_lower(text):
    return text.lower()

# Remove punctuation
punctuation = '!"#$&\'()*+-:;<=>?@[\\]^_`{|}~1234567890.'
def remove_punct(text):
    return "".join([char for char in text if char not in punctuation])

# Preprocess indications data
indications['Drug Indications'] = indications['Drug Indications'].astype(str)
indications['Drug Indications'] = indications['Drug Indications'].apply(lambda x: text_lower(x))
indications['Drug Indications'] = indications['Drug Indications'].apply(lambda x: remove_punct(x))

# Create a set of indications for faster matching
indications_set = set(indications['Drug Indications'])

# Preprocess the product label text
product_label_test_gpt['indications_and_usage'] = product_label_test_gpt['indications_and_usage'].astype(str)
product_label_test_gpt['indications_and_usage'] = product_label_test_gpt['indications_and_usage'].apply(lambda x: text_lower(x))
product_label_test_gpt['indications_and_usage'] = product_label_test_gpt['indications_and_usage'].apply(lambda x: remove_punct(x))

# Function to match drug indications in the original text
def match_indication(text):
    # Check for partial matches in the text
    matches = [indication for indication in indications_set if indication in text]
    # Return the matched indications as a comma-separated string
    return ', '.join(matches) if matches else 'No match'

# Apply the matching function to the original data
product_label_test_gpt['indication'] = product_label_test_gpt['indications_and_usage'].apply(lambda x: match_indication(x))


In [ ]:
# Not good...

product_label_test_gpt.head(30)